In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Загрузка данных
data = pd.read_csv('healthcare_dataset.csv')

# Вывод общей информации о данных
print("Общая информация о данных:")
print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("\nТипы данных по столбцам:")
print(data.dtypes)

Общая информация о данных:
Количество строк: 10000
Количество столбцов: 15

Типы данных по столбцам:
Name                   object
Age                     int64
Gender                 object
Blood Type             object
Medical Condition      object
Date of Admission      object
Doctor                 object
Hospital               object
Insurance Provider     object
Billing Amount        float64
Room Number             int64
Admission Type         object
Discharge Date         object
Medication             object
Test Results           object
dtype: object


In [11]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

# Загрузка данных
data = pd.read_csv('healthcare_dataset.csv')
data = data.drop('Name', axis=1)

# Предположим, что мы хотим предсказать тип медицинского состояния (Medical Condition)
X = data.drop('Medical Condition', axis=1)
y = data['Medical Condition']
X = pd.get_dummies(X)

# Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение параметров для поиска
param_grid = {
    'iterations': [100, 200,],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 6],
    'loss_function': ['MultiClass']
}

# Создание объекта GridSearchCV для модели
grid_search = GridSearchCV(
    estimator=CatBoostClassifier(verbose=False),  # Установка verbose=False,чтобы не было видно как обучается модель 
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)


# Подгонка модели к данным
grid_search.fit(X_train, y_train)

# Получение лучших параметров и оценщика с лучшими параметрами
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f'Лучшие параметры: {best_params}')

# Получение прогнозов с использованием лучшей модели
y_pred = best_model.predict(X_test)

# Вычисление и вывод точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

# Оценка производительности модели
report = classification_report(y_test, y_pred, zero_division=1)
print(f'Отчет о классификации:\n{report}')

# Определение отсутствующих классов в прогнозах
classes = np.unique(y_test)
predicted_classes = np.unique(y_pred)


Лучшие параметры: {'depth': 3, 'iterations': 100, 'learning_rate': 0.01, 'loss_function': 'MultiClass'}
Точность модели: 0.1605
Отчет о классификации:
              precision    recall  f1-score   support

   Arthritis       0.14      0.00      0.01       319
      Asthma       0.16      0.88      0.27       332
      Cancer       0.25      0.01      0.02       363
    Diabetes       1.00      0.00      0.00       303
Hypertension       0.15      0.07      0.09       336
     Obesity       1.00      0.00      0.00       347

    accuracy                           0.16      2000
   macro avg       0.45      0.16      0.07      2000
weighted avg       0.45      0.16      0.07      2000



Программа реализует загрузку и выбранного мною датасета,предварительно удалю столбец "Name" так как он не влияет никак на значения,разделяю данные на признаки,так как в датаесети имеются категоральные типы данных кодируем их с помощью One-Hot Encoding,разделению на обучающий и тестовый набор данных в соотношении 80/20,создаю 2 модели CatBoostClassifier (с разными гиперпараметрами), чтобы можно было проанализировать изменения в результате.Для каждой модели измеряется точность и выводится classification_report,также проводится кросс-валидация в 5 фолдов,её тоже вывел со средним значением.